# Deliverable 1

In [1]:
# Import Dependencies
import pandas as pd
import requests
import random
import numpy as np

# Import API keys
from config import weather_api_key

In [2]:
# Generate 2000 random latitudes and longitudes
lats = np.random.uniform(-90.000, 90.000, size = 2000)
lngs = np.random.uniform(-180.000, 180.000, size = 2000)
lat_lngs = zip(lats, lngs)
lat_lngs

In [3]:
# Add the latitudes and longitudes to a list (unzip)
coordinates = list(lat_lngs)

In [4]:
# Display the 2000 coordinates
coordinates

[(-5.85284554035232, -3.288479209853392),
 (-7.5261458674890065, 9.408350274335447),
 (29.926156065121674, -39.7577771921292),
 (9.554719506968851, 164.587412737818),
 (-37.633225389179856, 59.92921555984802),
 (71.79039149248109, 142.62349766797126),
 (79.83934850692049, -8.963033080469614),
 (73.76347924603681, -42.824637701833836),
 (-60.6814429109601, -167.73223210226817),
 (76.8357358268733, -37.96478558245596),
 (79.64798648168633, -101.11600403932),
 (32.357720078940545, -145.29064248211333),
 (-62.226768781045266, 87.16366376107158),
 (-54.29176776390924, 129.10412556830147),
 (-69.19898826703009, -106.95697389351277),
 (64.20705289416907, -152.41069025664027),
 (60.26529285145372, -22.702468138674192),
 (-24.526499319380903, 3.391648718003239),
 (68.78839363689653, -15.504138011050003),
 (-78.41090363962732, 94.94250671238785),
 (88.22100727877185, 25.095949930865174),
 (33.155513841534145, 93.4861474800544),
 (20.588610177090885, 178.24051907230313),
 (-52.181508157272006, -1

In [5]:
# Get the nearest City using the Citipy module
#import citipy
from citipy import citipy

# Create a list for the cities
cities = []

# identify the nearest city for each latitude and longitude combination
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name
    
    # if the city is unique, add it to the cities list.
    if city not in cities:
        cities.append(city)

# Print the number of cities.
len(cities)

751

In [6]:
# Perform API call to OpenWeatherMap to get:latitiude and longitude, max temp, 
# percent humidity, percent cloudiness, wind speed, and weather description

# import the API key
from config import weather_api_key

# Define Base URL
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&appid=" + weather_api_key

In [7]:
# Test CITY API call
city_url = url + "&q=" + "London"
test_weather = requests.get(city_url).json()


In [8]:
test_weather

{'coord': {'lon': -0.1257, 'lat': 51.5085},
 'weather': [{'id': 804,
   'main': 'Clouds',
   'description': 'overcast clouds',
   'icon': '04n'}],
 'base': 'stations',
 'main': {'temp': 47.89,
  'feels_like': 44.85,
  'temp_min': 45.86,
  'temp_max': 49.24,
  'pressure': 1020,
  'humidity': 79},
 'visibility': 10000,
 'wind': {'speed': 6.6, 'deg': 235, 'gust': 19.77},
 'clouds': {'all': 100},
 'dt': 1643847201,
 'sys': {'type': 2,
  'id': 2019646,
  'country': 'GB',
  'sunrise': 1643873760,
  'sunset': 1643907154},
 'timezone': 0,
 'id': 2643743,
 'name': 'London',
 'cod': 200}

In [9]:
test_weather["weather"][0]["description"]

'overcast clouds'

In [10]:
# Import the time library and the datetime modeul from the datetime library
import time
from datetime import datetime

# Create list to hold the weather data
city_data = []

# Print the beginning of the logging
print("Beginning Data Retrieval      ")
print("------------------------------")

# Create counters
record_count = 1
set_count = 1

# Loop through all the cities in the list
for i, city in enumerate(cities):
    
    # Group cities in sets of 50 for logging purposes
    if (i % 50 == 0 and i >=50):
        set_count += 1
        record_count = 1
        time.sleep(60)
    
    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")
    
    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count
    record_count += 1

# Run an API request for each of the cities

    try:
        #Parse the JSON and retrieve data
        city_weather = requests.get(city_url).json()
        # Parse out the needed data
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_description = city_weather["weather"][0]["description"]
        # Convert the date to ISO standard
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        #Append the city information into city_data list
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Desription": city_description,
                          "Date": city_date})
    # If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass
# Indicate that Data Loading is complete.
print("--------------------------")
print("Data Retieval Complete     ")
print("--------------------------")

#  Convert the array of dictionaires to a Pandas DataFrame
city_data_df= pd.DataFrame(city_data)
city_data_df.head(10)

# Create the output file (CSV)
#output_data_file = "Weather_Database/WeatherPy_Database.csv"
output_data_file = "WeatherPy_Database.csv"
# Export the City_data into a CSV
city_data_df.to_csv(output_data_file, index_label = "City_ID")

# Deliverable 2

In [11]:
# IMPORT CSV TO CREATE AVOID THE NEED TO REPEAT THE API CALL
file_in = "WeatherPy_Database.csv"
city_data_df = pd.read_csv(file_in)

city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Desription,Date
0,0,Charagua,-19.8000,-63.2167,89.17,48,58,8.23,broken clouds,2022-02-02 22:30:46
1,1,Ushuaia,-54.8000,-68.3000,44.26,57,75,25.32,broken clouds,2022-02-02 22:30:46
2,2,Mataura,-46.1927,168.8643,58.48,97,100,3.96,moderate rain,2022-02-02 22:28:57
3,3,Winslow,35.0242,-110.6974,41.18,23,100,5.01,overcast clouds,2022-02-02 22:30:47
4,4,Lebu,-37.6167,-73.6500,63.72,79,45,11.01,scattered clouds,2022-02-02 22:29:34
...,...,...,...,...,...,...,...,...,...,...
688,688,Smoky Lake,54.1169,-112.4686,-10.44,70,97,11.50,overcast clouds,2022-02-02 22:43:11
689,689,Traralgon,-38.1833,146.5333,64.42,64,92,7.14,overcast clouds,2022-02-02 22:47:50
690,690,Pindushi,62.9152,34.5800,5.04,98,95,3.31,overcast clouds,2022-02-02 22:47:51
691,691,Galesong,-5.3166,119.3661,75.22,81,100,3.51,overcast clouds,2022-02-02 22:47:51
